In [1]:
import numpy as np
import os
import sys
import imageio
#import skimage.transform
from pathlib import Path

from llff.poses.colmap_wrapper import run_colmap
import llff.poses.colmap_read_model as read_model

In [11]:
#dir = Path('/home/nikola/projects/pxo/data/prepared/nerfstudio/living_room/colmap/')
dir = Path('/home/nikola/projects/pxo/data/prepared/nerfstudio/living_room_llff/')

### Original `load_colmap_data()` functon from poses_utils.py

In [12]:
def load_colmap_data(realdir):
    
    camerasfile = os.path.join(realdir, 'sparse/0/cameras.bin')
    camdata = read_model.read_cameras_binary(camerasfile)
    
    # cam = camdata[camdata.keys()[0]]
    list_of_keys = list(camdata.keys())
    cam = camdata[list_of_keys[0]]
    print( 'Cameras', len(cam))

    h, w, f = cam.height, cam.width, cam.params[0]
    # w, h, f = factor * w, factor * h, factor * f
    hwf = np.array([h,w,f]).reshape([3,1])
    
    imagesfile = os.path.join(realdir, 'sparse/0/images.bin')
    imdata = read_model.read_images_binary(imagesfile)
    
    w2c_mats = []
    bottom = np.array([0,0,0,1.]).reshape([1,4])
    
    names = [imdata[k].name for k in imdata]
    print( 'Images #', len(names))
    perm = np.argsort(names)
    for k in imdata:
        im = imdata[k]
        R = im.qvec2rotmat()
        t = im.tvec.reshape([3,1])
        m = np.concatenate([np.concatenate([R, t], 1), bottom], 0)
        w2c_mats.append(m)
    
    w2c_mats = np.stack(w2c_mats, 0)
    c2w_mats = np.linalg.inv(w2c_mats)
    
    poses = c2w_mats[:, :3, :4].transpose([1,2,0])
    poses = np.concatenate([poses, np.tile(hwf[..., np.newaxis], [1,1,poses.shape[-1]])], 1)
    
    points3dfile = os.path.join(realdir, 'sparse/0/points3D.bin')
    pts3d = read_model.read_points3d_binary(points3dfile)
    
    # must switch to [-u, r, -t] from [r, -u, t], NOT [r, u, -t]
    poses = np.concatenate([poses[:, 1:2, :], poses[:, 0:1, :], -poses[:, 2:3, :], poses[:, 3:4, :], poses[:, 4:5, :]], 1)
    
    return poses, pts3d, perm

In [13]:
poses, pts3d, perm = load_colmap_data(dir)
poses.shape

Cameras 5
Images # 606


(3, 5, 606)

### Original `save_poses()` functon from poses_utils.py

In [20]:
def save_poses(basedir, poses, pts3d, perm):
    pts_arr = []
    vis_arr = []
    for k in pts3d:
        pts_arr.append(pts3d[k].xyz)
        cams = [0] * poses.shape[-1]
        for ind in pts3d[k].image_ids:
            if len(cams) < ind - 1:
                print('ERROR: the correct camera poses for current points cannot be accessed')
                return
            cams[ind-1] = 1
        vis_arr.append(cams)

    pts_arr = np.array(pts_arr)
    vis_arr = np.array(vis_arr)
    print( 'Points', pts_arr.shape, 'Visibility', vis_arr.shape )
    
    zvals = np.sum(-(pts_arr[:, np.newaxis, :].transpose([2,0,1]) - poses[:3, 3:4, :]) * poses[:3, 2:3, :], 0)
    valid_z = zvals[vis_arr==1]
    print( 'Depth stats', valid_z.min(), valid_z.max(), valid_z.mean() )
    
    save_arr = []
    for i in perm:
        vis = vis_arr[:, i]
        zs = zvals[:, i]
        zs = zs[vis==1]
        close_depth, inf_depth = np.percentile(zs, .1), np.percentile(zs, 99.9)
        print(i, zs)
        #print( i, close_depth, inf_depth )
        
        save_arr.append(np.concatenate([poses[..., i].ravel(), np.array([close_depth, inf_depth])], 0))
    save_arr = np.array(save_arr)
    print( save_arr.shape )
    
    #np.save(os.path.join(basedir, 'poses_bounds.npy'), save_arr)

In [21]:
save_poses(dir, poses, pts3d, perm)

Points (36516, 3) Visibility (36516, 606)
Depth stats 0.2951876655685503 103.90463190874928 5.737001129842842
0 [2.59528949 2.47187485 2.3166243  2.51667513 3.28850797 3.19511401
 2.40316191 3.27902115 2.53156095 2.37903163 2.53454166 2.3782546
 2.53260177 2.54433658 2.59132705 2.5612325  2.54251911 2.40019
 2.5142252  2.39382513 2.36346692 2.5129915  2.37230109 2.60829397
 2.37400279 2.51948182 2.36249029 2.51076972 2.37688109 3.35284158
 3.63337566 3.73356114 2.28751611 2.59686776 2.59686776 2.45643757
 2.57617268 2.55415463 2.58481237 2.56306485 2.56935617 2.49255626
 2.53842942 2.49525237 2.479539   2.479539   2.50975315 2.26448981
 2.56014516 2.59326501 3.88226544 2.60095038 2.59409934 2.51866563]
1 [2.31047544 2.28978105 2.39138819 2.37794575 2.32642136 2.37056654
 2.13872385 2.33164297 2.35144527 2.09730619 2.1952323  2.35253436
 2.33848155 2.35110198 2.68633277 2.36922252 2.39760312 2.36257666
 2.33768016 2.18255787 2.3365821  2.18927164 2.4108871  2.32695006
 2.18103753 2.3341